Professor Sourav Chatterjee's xicor coefficient of correlation (<a href="https://win-vector.com/2021/12/26/how-to-read-sourav-chatterjees-basic-xicor-defenition/">our note</a>, <a href="https://doi.org/10.1080/01621459.2020.1758115">JASA</a>; original sources: <a href="https://CRAN.R-project.org/package=XICOR">R package</a>, <a href="https://arxiv.org/abs/1909.10140">Arxiv</a>, <a href="https://news.ycombinator.com/item?id=29687613">Hacker News</a>, and <a href="https://github.com/czbiohub/xicor">a Python package</a> (different author)).

In [ ]:
import pandas as pd
import numpy as np
from data_algebra.data_ops import *

def xicor(x, y):
    d = pd.DataFrame({'x': x, 'y': y})
    n = d.shape[0]
    d['x_tie_breaker'] = np.random.choice(n, size=n, replace=False)
    d['y_tie_breaker'] = np.random.choice(n, size=n, replace=False)
    res_frame = ex(
        data(d=d)
            .extend(
                {'r': '(1).cumsum()'},
                order_by=['y', 'y_tie_breaker'])
            .extend(
                {'l': '(1).cumsum()'},
                order_by=['y', 'y_tie_breaker'],
                reverse=['y', 'y_tie_breaker'])
            .extend(
                {'rplus': 'r.shift()'},
                order_by=['x', 'x_tie_breaker'],
                reverse=['x', 'x_tie_breaker'])
            .extend({'n': '(1).sum()'})
            .extend(
                {
                    'rdiff': '(rplus - r).abs().coalesce_0()',
                    'lterm': 'l * (n - l)',
                })
            .project(
                {
                    'num_sum': 'rdiff.sum()',
                    'den_sum': 'lterm.sum()',
                    'n': 'n.max()',  # pseudo-aggregation, column is constant
                }
                )
            .extend({'xicor': '1 - n * num_sum / ( 2 * den_sum)'})
            .select_columns('xicor')
    )
    return res_frame['xicor'].values[0]
